In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1588716/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_1588716/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_4_1_7,0.821703,-0.006755,-1.668881,0.838278,0.432791,2.018231,1.968067,0.467646,1.245572e+14,0.657869,1.345788,0.668230,195.674999,406.590564,"Hidden Size=[24], regularizer=0.02, learning_r..."
1,model_4_2_2,0.837923,-0.010385,-1.673742,0.792140,0.393419,2.025507,1.971652,0.265538,1.187868e+14,0.627231,1.314331,0.637109,195.865761,406.781326,"Hidden Size=[24], regularizer=0.02, learning_r..."
2,model_4_1_2,0.805686,-0.011812,-1.687732,0.822048,0.471669,2.028370,1.981969,0.514578,1.303499e+14,0.686782,1.376851,0.697598,195.502956,406.418521,"Hidden Size=[24], regularizer=0.02, learning_r..."
3,model_4_1_5,0.816022,-0.014244,-1.691850,0.832285,0.446582,2.033244,1.985005,0.484976,1.263175e+14,0.668268,1.356806,0.678792,195.612266,406.527831,"Hidden Size=[24], regularizer=0.02, learning_r..."
4,model_4_2_6,0.853185,-0.015479,-1.683523,0.812412,0.356374,2.035719,1.978864,0.239641,1.128451e+14,0.596971,1.284733,0.606372,196.063549,406.979115,"Hidden Size=[24], regularizer=0.02, learning_r..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,model_1_49_9,0.972922,-0.397924,-2.206772,0.967233,0.065727,2.802403,2.364715,0.050538,5.921023e+13,0.256373,0.963128,0.260411,39.444480,76.409064,"Hidden Size=[4], regularizer=0.02, learning_ra..."
510,model_3_2_3,0.839490,-0.423982,-2.833845,0.851010,0.389615,2.854641,2.827127,0.287466,9.900103e+13,0.624192,0.315158,0.634022,99.885190,206.430167,"Hidden Size=[12], regularizer=0.02, learning_r..."
511,model_3_2_0,0.827362,-0.454730,-2.924703,0.842136,0.419054,2.916282,2.894127,0.304589,1.016832e+14,0.647344,0.263413,0.657539,99.739513,206.284489,"Hidden Size=[12], regularizer=0.02, learning_r..."
512,model_3_1_1,0.804070,-0.591485,-3.289570,0.609845,0.475593,3.190434,3.163184,0.520606,1.005484e+14,0.689633,0.164031,0.700494,99.486384,206.031361,"Hidden Size=[12], regularizer=0.02, learning_r..."
